# Explainability with Grad-CAM

Visualize class-discriminative regions on histology patches using Grad-CAM for a CNN backbone.

In [ ]:
# Minimal Grad-CAM on a timm model
import importlib
if importlib.util.find_spec('timm') is None:
    get_ipython().system('pip -q install timm')
import torch, timm, torchvision.transforms as T
from PIL import Image
import io, requests, numpy as np
model = timm.create_model('resnet50', pretrained=True)
model.eval()
target_layer = model.layer4[-1].conv3
preprocess = T.Compose([T.Resize(256), T.CenterCrop(224), T.ToTensor(), T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])])
def load_img():
    url = 'https://picsum.photos/seed/histo3/256'
    try:
        return Image.open(io.BytesIO(requests.get(url, timeout=5).content)).convert('RGB')
    except Exception:
        return Image.fromarray((np.random.rand(256,256,3)*255).astype('uint8'), 'RGB')
img = load_img()
x = preprocess(img).unsqueeze(0)
# Forward hook to get activations and gradients
acts = {}; grads = {}
def fwd_hook(m, i, o): acts['value'] = o.detach()
def bwd_hook(m, gi, go): grads['value'] = go[0].detach()
h1 = target_layer.register_forward_hook(fwd_hook)
h2 = target_layer.register_full_backward_hook(bwd_hook)
logits = model(x)
cls = logits.argmax(dim=1)
model.zero_grad(); logits[0, cls].backward()
w = grads['value'].mean(dim=(2,3), keepdim=True)
cam = (w * acts['value']).sum(dim=1).squeeze().numpy()
cam = (cam - cam.min()) / (cam.max() - cam.min() + 1e-6)
print('CAM shape:', cam.shape)
h1.remove(); h2.remove()